In [353]:
import pandas as pd
import numpy as np
import math
import json

In [354]:
df = pd.read_excel("data/all_products_19_12_2.xlsx")

In [355]:
df = df.drop(["grouped_data_sheets", "Unnamed: 0.1"], axis=1)

In [356]:
# Filter the DataFrame where 'is_range' is 1
df_filtered = df[df['is_range'] == 1]

# Group by 'product_family_name' and aggregate 'Data sheets'
# Group by 'product_family_name' and aggregate 'Data sheets'
grouped = df_filtered.groupby('product_family_name')['Product_Name'].apply(lambda x: list(set(x)))
#print grouped line 1 
# print(grouped[0])
# Convert the grouped Series into a DataFrame
grouped_df = grouped.reset_index()
grouped_df.columns = ['product_family_name', 'grouped_data_sheets']

# Merge the grouped_df into the original df
df_with_group = pd.merge(df, grouped_df, on='product_family_name', how='left')

# change the order of the columns to add the new column 'grouped_data_sheets' right after is_range
cols = df_with_group.columns.tolist()
cols.insert(13, cols.pop(cols.index('grouped_data_sheets')))
df_with_group = df_with_group.reindex(columns= cols)



In [357]:
for i in df_with_group.index:
    if type(df_with_group.at[i, 'grouped_data_sheets']) == float:
        df_with_group.at[i, 'grouped_data_sheets'] = []
    if type(df_with_group.at[i, 'related']) == float:
        df_with_group.at[i, 'related'] = []
    else:
        df_with_group.at[i, 'related'] = list(json.loads(df_with_group.at[i, 'related']).keys())


In [358]:
df_with_group.at[i, 'related']

[]

In [359]:
df_with_group["grouped_data_sheets"] = df_with_group.apply(lambda row: json.dumps({"productRange": row["grouped_data_sheets"], "relatedProducts": row["related"]}), axis=1)

In [360]:
df_with_group["grouped_data_sheets"][1]

'{"productRange": ["38 kHz Single-beam transducer", "500 KHZ TRANSDUCER", "120 kHz single-beam transducer", "18 kHz single-beam transducer", "Combi 50/200 D", "Transducer 15 - 17", "Simrad 38/200 D", "SIDESCAN 500 KHZ", "200 kHz single-beam transducer in robust housing", "12 kHz dual-beam transducer", "Simrad 38-9", "50 kHz Single-beam transducer", "TRANSDUCER 200-3G"], "relatedProducts": ["Simrad 38-7"]}'

In [361]:
df_with_group = df_with_group.drop(columns=["product_family_name", "is_range", "related"])

In [362]:
df_with_group.to_excel("data/all_products_19_12.xlsx", index=False)